In [15]:
import pandas as pd
import numpy as np
import string
import spacy
import multiprocessing 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [2]:
# nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner', 'textcat'])
nlp = spacy.load('en_vectors_web_lg', disable=['parser', 'tagger', 'ner', 'textcat'])

In [3]:
dataFile = "../data/cleaned.csv"
data1 = pd.read_csv(dataFile)
data2 = pd.read_csv('../data/hate_cleaned.csv')

In [4]:
data2 = data2.drop(data2[data2['Class'] == 'neither'].index)
data2['Class'] = 1

In [ ]:
data[data.isnull().any(axis=1)]

In [ ]:
data[data['Class'] == 1].tail(n=10)

In [5]:
ent_whitelist = set(['ADJ', 'ADV', 'PROPN', 'VERB', 'NOUN', 'INTJ', 'PRON', 'X', 'NUM'])
whitelist = set(string.ascii_lowercase + string.digits)

In [6]:
megaSpam = pd.read_csv('../data/polluters.tsv', sep='\t')
megaNonSpam = pd.read_csv('../data/legit.tsv', sep='\t')

In [7]:
megaSpam['Class'] = 1
megaNonSpam['Class'] = 0

In [8]:
data = pd.concat([data1, data2, megaSpam, megaNonSpam])
data.head()

/home/will/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Class,FavCount,RtCount,ScreenName,Text,TweetID,Verified
0,0,0.0,8.0,Tactical_Things,RT @HouseOfTraitors: FARAGE 12 - 0 Mandelsc...,732588910078001152,1.0
1,0,0.0,0.0,kataisa,@TheBriefing2016 @HillaryClinton Can Hillary s...,732588910073778176,1.0
2,0,0.0,5423.0,Josh_Ward_,RT @HQUESTED: Harvey price just said cunt on n...,732588910061232128,1.0
3,0,0.0,0.0,networkMCI,@xonighttimexo I will put them in a Safety dep...,732588910056902656,1.0
4,0,0.0,0.0,nbharper,"you should listen, but you probably won't bc y...",732588910069620736,1.0


In [ ]:
data.tail()

In [ ]:
data['Text'].shape[0] // 50

In [9]:
%%time

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(len(X_test), len(y_test))
print(len(X_train), len(y_train))

# classifier = GradientBoostingClassifier()
classifier = XGBClassifier
# classifier = LogisticRegression()
classifier.fit(X_train, y_train)

y_hat = classifier.predict(X_test)


Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
Found error: nan, <class 'float'>
CPU times: user 29.9 s, sys: 16.7 s, total: 46.6 s
Wall time: 2min 14s


In [ ]:
cleanData = [' '.join([x.lower() for x in y.split(' ') if len(x) > 0 and x[0] != '@']) for y in cleanData]

In [10]:
X = np.array(cleanData)
y = data['Class']
X.shape

(5638816, 300)

In [16]:
%%time

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(len(X_test), len(y_test))
print(len(X_train), len(y_train))

# classifier = GradientBoostingClassifier()
# classifier = XGBClassifier(n_jobs=30)
classifier = MLPClassifier(hidden_layer_sizes=(100, 200, 100), activation='logistic', early_stopping=True)
# classifier = LogisticRegression()
classifier.fit(X_train, y_train)

y_hat = classifier.predict(X_test)


1409704 1409704
4229112 4229112
CPU times: user 1d 5min 2s, sys: 1min 58s, total: 1d 7min
Wall time: 1h 36min 25s


In [17]:
import pickle
with open('thepickle_NN.pkl', 'bw') as f:
#     pickle.dump([encoder, vectorizer, classifier], f)
    pickle.dump(classifier, f)

In [ ]:
cleanData[:100]
for d in cleanData:
    if 'null' in d:
        print(d)

In [ ]:
# vectorizer = CountVectorizer()
# for i in range(0, len(cleanData) - 9999, 10000):
# #     X = vectorizer.fit_transform(cleanData[i:i+10000])
    

In [ ]:


X.shape
# X.toarray().shape

In [ ]:
# encoder = LabelEncoder()
# y = encoder.fit_transform(data["Class"])
y = data['Class']

In [ ]:
print(y.shape)
print(len(cleanData))

In [ ]:
y = y

In [ ]:
clean_train, clean_test, y_train, y_test = train_test_split(cleanData, y, test_size=0.25, random_state=42)
print(len(clean_test), len(y_test))
print(len(clean_train), len(y_train))
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(clean_train)

# classifier = GradientBoostingClassifier()
classifier = LogisticRegression()
classifier.fit(X_vec, y_train)

y_vec = vectorizer.transform(clean_test)
y_hat = classifier.predict(y_vec)


In [ ]:
classifier = GradientBoostingClassifier()
classifier.fit(X, y)

In [14]:
from collections import Counter
vals = Counter(y)
print(vals)
print(vals[0] / len(y), vals[1] / len(y))
# Counter(y)

Counter({0: 3300334, 1: 2338482})
0.5852884719061591 0.414711528093841


In [18]:
accuracy = accuracy_score(y_test, y_hat)
accuracy

0.8149249771583255

In [ ]:
accuracy = accuracy_score(y_test, y_hat)
accuracy

# PICKLE EVERYTHING

In [ ]:
import pickle
with open('thepickle_LR_wordvec.pkl', 'bw') as f:
#     pickle.dump([encoder, vectorizer, classifier], f)
    pickle.dump(classifier, f)

In [ ]:
coefs = classifier.coef_[0]

In [ ]:
k = 100
top_k = np.argpartition(coefs, -k)[-k:]
top_k_sorted=top_k[np.argsort(coefs[top_k])]

In [ ]:
feats = vectorizer.get_feature_names()
[feats[x] for x in top_k_sorted]